## import

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install xgboost
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

## input dataset

In [ ]:
df = pd.read_csv('/gdrive/My Drive/Colab Notebooks/data_no_multicol_female.txt', delimiter='\t')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359 entries, 1 to 462
Columns: 338 entries, cluster_assignment_sub to A8d_.1.tell_professional
dtypes: float64(159), int64(179)
memory usage: 950.8 KB
None


In [ ]:
df = pd.read_csv('/gdrive/My Drive/Colab Notebooks/data_no_multicol_female.txt', delimiter='\t')
data=df.iloc[:,1:338]
target=df.iloc[:,0]
target= target.replace(1,0)
target= target.replace(2,1)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359 entries, 1 to 462
Columns: 338 entries, cluster_assignment_sub to A8d_.1.tell_professional
dtypes: float64(159), int64(179)
memory usage: 950.8 KB
None


In [ ]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359 entries, 1 to 462
Columns: 337 entries, CHILDREN to A8d_.1.tell_professional
dtypes: float64(159), int64(178)
memory usage: 948.0 KB
None


## xgboost

In [ ]:
# Initialize an array to store feature importance for each iteration
import numpy as np
feature_importances = np.zeros((50, len(data.columns)))

In [ ]:
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
# repeat 50 times
for i in range(50):
    train_x, test_x, train_y, test_y = train_test_split(data,target,test_size=0.2,random_state=(i*i+5))
    model = XGBClassifier(
        n_estimators=25,
        max_depth=5,
        learning_rate= 0.5,
        subsample=0.75,
        colsample_bytree=0.75,
        reg_lambda=10,
        eval_metric='auc',
        early_stopping_rounds=5,
        random_state=i
    )
    model.fit(train_x, train_y, eval_set=[(test_x, test_y)])
    #perm = PermutationImportance(model, random_state=101).fit(test_x, test_y)
    feature_importances[i, :] = model.feature_importances_

[0]	validation_0-auc:0.74959
[1]	validation_0-auc:0.79261
[2]	validation_0-auc:0.81331
[3]	validation_0-auc:0.80235
[4]	validation_0-auc:0.81372
[5]	validation_0-auc:0.82468
[6]	validation_0-auc:0.83523
[7]	validation_0-auc:0.83036
[8]	validation_0-auc:0.84334
[9]	validation_0-auc:0.83847
[10]	validation_0-auc:0.83929
[11]	validation_0-auc:0.83929
[12]	validation_0-auc:0.84659
[13]	validation_0-auc:0.84578
[14]	validation_0-auc:0.84334
[15]	validation_0-auc:0.85146
[16]	validation_0-auc:0.85308
[17]	validation_0-auc:0.84984
[18]	validation_0-auc:0.84903
[19]	validation_0-auc:0.84903
[20]	validation_0-auc:0.85227
[21]	validation_0-auc:0.85714
[22]	validation_0-auc:0.85471
[23]	validation_0-auc:0.85227
[24]	validation_0-auc:0.85308
[0]	validation_0-auc:0.68394
[1]	validation_0-auc:0.74790
[2]	validation_0-auc:0.77544
[3]	validation_0-auc:0.77871
[4]	validation_0-auc:0.80579
[5]	validation_0-auc:0.82446
[6]	validation_0-auc:0.82446
[7]	validation_0-auc:0.83380
[8]	validation_0-auc:0.82446

## permutation importance

# #result

In [ ]:
# Calculate average feature importance
average_importances = np.mean(feature_importances, axis=0)

# Sort and get the index of the top 5 features
top_10_feature_indices = np.argsort(average_importances)[-10:]

# Get and print the top 5 most important feature names and their average importance
top_features = data.columns[top_10_feature_indices]
for feature, importance in zip(top_features, average_importances[top_10_feature_indices]):
    print(f"Feature: {feature}, Importance: {importance}")

Feature: DEATH_WORST, Importance: 0.021235839715227484
Feature: PHQ_3, Importance: 0.02232326235389337
Feature: AGE_ONSET_MDD, Importance: 0.02637564345262945
Feature: Smokebefore, Importance: 0.029336199853569268
Feature: GAD_2, Importance: 0.03173003817209974
Feature: PHQ_4, Importance: 0.03256162352859974
Feature: Q4_Currently_Smoke, Importance: 0.03273241452872753
Feature: PHQ_7, Importance: 0.033935848489782074
Feature: GAD_3, Importance: 0.04046430564136244
Feature: GAD_4, Importance: 0.05928625582717359
